In [1]:
# Set country to ADB country code of country
country = 'RUS'

In [2]:
# Import libraries and functions
import wbdata as wb                                       
import pandas as pd
import numpy as np                                     
import datetime as dt                                     
import docx                                               
from docx.shared import Cm       
from docx.shared import Pt                         
from docx.enum.text import WD_ALIGN_PARAGRAPH             
from matplotlib import pyplot as plt 
from matplotlib import ticker
import random
import Haver
import calendar
import re
from pandasgui import show

In [3]:
# Create a dictionary of all indicators to be scraped from the World Bank API
all_indicators = {'NY.GDP.MKTP.KD.ZG': 'GDP growth (%)',
                  'NE.CON.PRVT.ZS': 'Private consumption (% of GDP)', 
                  'NE.CON.GOVT.ZS': 'Government expenditure (% of GDP)',
                  'NE.GDI.TOTL.ZS': 'Gross capital formation (% of GDP)', 
                  'NE.EXP.GNFS.ZS': 'Exports (% of GDP)',
                  'NE.IMP.GNFS.ZS': 'Imports (% of GDP)',
                  'NE.CON.PRVT.KD.ZG': 'Private consumption (annual growth, %)', 
                  'NE.CON.GOVT.KD.ZG': 'Government expenditure (annual growth, %)',
                  'NE.GDI.TOTL.KD.ZG': 'Gross capital formation (annual growth, %)',
                  'NE.EXP.GNFS.KD.ZG': 'Exports (annual growth, %)',
                  'NE.IMP.GNFS.KD.ZG': 'Imports (annual growth, %)',
                  'NE.EXP.GNFS.CD': 'Exports (current prices, USD)',
                  'NE.IMP.GNFS.CD': 'Imports (current prices, USD)',
                  'NV.AGR.TOTL.ZS': 'Agriculture (% of GDP)',
                  'NV.AGR.TOTL.KD.ZG': 'Agriculture (annual growth, %)',
                  'NV.IND.TOTL.ZS': 'Industry (including construction) (% of GDP)',
                  'NV.IND.TOTL.KD.ZG': 'Industry (including construction) (annual growth, %)',
                  'NV.SRV.TOTL.ZS': 'Services (% of GDP)',
                  'NV.SRV.TOTL.KD.ZG': 'Services (annual growth, %)',
                  'SL.UEM.TOTL.NE.ZS': 'Unemployment rate (%)',
                  'FP.CPI.TOTL.ZG': 'Inflation rate (%)',
                  'BN.CAB.XOKA.GD.ZS': 'Current account balance (% of GDP)',
                  'BN.GSR.GNFS.CD': 'Net trade in goods and services (current USD)',
                  'BN.GSR.MRCH.CD': 'Net trade in goods (current USD)',
                  'FI.RES.TOTL.CD': 'Total reserves (includes gold, current USD)',
                  'FI.RES.TOTL.MO': 'Total reserves in months of imports'}

# Set the time period
data_date = dt.datetime(dt.date.today().year - 6, 1, 1), dt.datetime(dt.date.today().year - 1, 1, 1)

# Read Excel file of country codes and country names
country_codes = pd.read_excel('all-data.xlsx', sheet_name='data', index_col='adb_code')

In [4]:
gdp_growth_quarterly_codes = {'ngpc': 'GDP growth',
                              'nchc': 'Household consumption',
                              'ncpc': 'Private consumption',
                              'ncgc': 'Government expenditure',
                              'nfbc': 'Gross capital formation',
                              'nxnc': 'Net exports',
                              'ngpy': 'GDP growth',
                              'ncpy': 'Household consumption',
                              'ncgy': 'Government expenditure',
                              'nfby': 'Gross capital formation',
                              'nxy': 'Exports',
                              'nmy': 'Imports'}
gdp_growth_quarterly_country = [str(country_codes.haver_code[country]) + item for item in gdp_growth_quarterly_codes.keys()]
gdp_growth_quarterly_indicators = ['h' + item for item in gdp_growth_quarterly_country] + ['s' + item for item in gdp_growth_quarterly_country] + ['n' + item for item in gdp_growth_quarterly_country]
df_gdp_growth_quarterly = Haver.data(gdp_growth_quarterly_indicators, database='emerge', dates=True)
if type(df_gdp_growth_quarterly) is not dict:
    df_gdp_growth_quarterly = df_gdp_growth_quarterly[[item for item in df_gdp_growth_quarterly if item.startswith('h')]]
    df_gdp_growth_quarterly = (df_gdp_growth_quarterly.pct_change(4, fill_method=None) * 100).round(1).tail(5).dropna()
    df_gdp_growth_quarterly.columns = [gdp_growth_quarterly_codes[item[-4:]] for item in df_gdp_growth_quarterly.columns]
elif df_gdp_growth_quarterly['codelists']['codesfound']:
    df_gdp_growth_quarterly = Haver.data(df_gdp_growth_quarterly['codelists']['codesfound'], dates=True)
    if len([item for item in df_gdp_growth_quarterly if item.startswith('h')]) > 0:
        df_gdp_growth_quarterly = df_gdp_growth_quarterly[[item for item in df_gdp_growth_quarterly if item.startswith('h')]]
        df_gdp_growth_quarterly = (df_gdp_growth_quarterly.pct_change(4, fill_method=None) * 100).round(1).tail(5).dropna()
        df_gdp_growth_quarterly.columns = [gdp_growth_quarterly_codes[item[-4:]] for item in df_gdp_growth_quarterly.columns]
    elif len([item for item in df_gdp_growth_quarterly if item.startswith('n')]) > 0:
        df_gdp_growth_quarterly = df_gdp_growth_quarterly[[item for item in df_gdp_growth_quarterly if item.startswith('n')]].round(1).tail(5).dropna()
        df_gdp_growth_quarterly.columns = [gdp_growth_quarterly_codes[re.sub('[0-9]+', '', item[-4:])] for item in df_gdp_growth_quarterly.columns]
    else:
        df_gdp_growth_quarterly = (df_gdp_growth_quarterly.pct_change(4, fill_method=None) * 100).round(1).tail(5).dropna()
        df_gdp_growth_quarterly.columns = [gdp_growth_quarterly_codes[item[-4:]] for item in df_gdp_growth_quarterly.columns]


retail_codes = {'trs': 'Retail sales index'}
retail_country = [str(country_codes.haver_code[country]) + item for item in retail_codes.keys()]
retail_indicators = ['n' + item for item in retail_country] + ['h' + item for item in retail_country]
df_retail = Haver.data(retail_indicators, database='emerge', dates=True)
if type(df_retail) is not dict:
    df_retail = df_retail[[item for item in df_retail.columns if item.startswith('h')]]
    df_retail = (df_retail.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
    df_retail.columns = [retail_codes[item[-3:]] for item in df_retail.columns]
elif df_retail['codelists']['codesfound']:
    df_retail = Haver.data(df_retail['codelists']['codesfound'], dates=True)
    df_retail = (df_retail.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
    df_retail.columns = [retail_codes[item[-3:]] for item in df_retail.columns]

consumer_codes = {'vcc': 'Consumer confidence index',
                  'vce': 'Consumer expectations index'}
consumer_country = [str(country_codes.haver_code[country]) + item for item in consumer_codes.keys()]
consumer_indicators = ['n' + item for item in consumer_country]
df_consumer = Haver.data(consumer_indicators, database='emerge', dates=True)
if type(df_consumer) is not dict:
    df_consumer.columns = [consumer_codes[item[-3:]] for item in df_consumer.columns]

production_codes =  {'d': 'Industrial production',
                     'dm': 'Manufacturing', 
                     'dn': 'Mining and quarrying',
                     'dvw': 'Water supply, sewerage, waste management & remediation',
                     'dvu': 'Electricity, gas, steam and air conditioning supply'}
production_country = [str(country_codes.haver_code[country]) + item for item in production_codes.keys()]
production_indicators = ['s' + item for item in production_country] + ['n' + item for item in production_country]
df_production = Haver.data(production_indicators, database='emerge', dates=True)
if type(df_production) is not dict:
    df_production = df_production[[item for item in df_production.columns if item.startswith('s')]]
    df_production = (df_production.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
    df_production.columns = [production_codes[''.join([i for i in (item[-3:]) if not i.isdigit()])] for item in df_production.columns]
elif df_production['codelists']['codesfound']:
    df_production = Haver.data(df_production['codelists']['codesfound'], dates=True)
    if len([item for item in df_production.columns if item.startswith('s')]) > 0:
        df_production = df_production[[item for item in df_production.columns if item.startswith('s')]]
        df_production = (df_production.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
        df_production.columns = [production_codes[''.join([i for i in (item[-3:]) if not i.isdigit()])] for item in df_production.columns]
    else:
        df_production = (df_production.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
        df_production.columns = [production_codes[''.join([i for i in (item[-3:]) if not i.isdigit()])] for item in df_production.columns]

cpi_codes = {'pc': 'Overall inflation', 
             'pcf': 'Food',
             'pch': 'Housing, rent, water, electricity, gas & other fuels',
             'pct': 'Transportation',
             'pcc': 'Communication',
             'pcm': 'Health/medical care',
             'pcr': 'Recreation',
             'pce': 'Education'}
cpi_country = [str(country_codes.haver_code[country]) + item for item in cpi_codes]
cpi_indicators = ['h' + item for item in cpi_country]
df_cpi = Haver.data(cpi_indicators, database='emerge', dates=True)
if type(df_cpi) is not dict:
    df_cpi = (df_cpi.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
    df_cpi.columns = [cpi_codes[''.join([i for i in (item[-3:]) if not i.isdigit()])] for item in df_cpi.columns]
elif df_cpi['codelists']['codesfound']:
    df_cpi = Haver.data(df_cpi['codelists']['codesfound'], dates=True)
    df_cpi = (df_cpi.pct_change(12, fill_method=None) * 100).round(1).tail(13).dropna()
    df_cpi.columns = [cpi_codes[''.join([i for i in (item[-3:]) if not i.isdigit()])] for item in df_cpi.columns]
    
policyrate_monthly_codes = {'rtar': 'Policy rate (EOP, monthly, %)'}
policyrate_monthly_country = [str(country_codes.haver_code[country]) + item for item in policyrate_monthly_codes]
policyrate_monthly_indicators = ['n' + item for item in policyrate_monthly_country]
df_policyrate_monthly = Haver.data(policyrate_monthly_indicators, database='emerge', dates=True)
if type(df_policyrate_monthly) is not dict:
    df_policyrate_monthly.columns = [policyrate_monthly_codes[item[-4:]] for item in df_policyrate_monthly.columns]

policyrate_annual_codes = {'ic': 'Policy rate (EOP, annual, %)'}
policyrate_annual_country = [str(country_codes.haver_code[country]) + item for item in policyrate_annual_codes]
policyrate_annual_indicators = ['c' + item for item in policyrate_annual_country]
df_policyrate_annual = Haver.data(policyrate_annual_indicators, database='ifsann', dates=True)
if type(df_policyrate_annual) is not dict:
    df_policyrate_annual.columns = [policyrate_annual_codes[item[-2:]] for item in df_policyrate_annual.columns]

In [5]:
# Scrape data from the World Bank API (country in [] because want to access the 'RUS' row in country_codes DF, alternative to iat, etc.)
df_all = wb.get_dataframe(indicators=all_indicators, country=country_codes.iso_code[country], data_date=data_date, convert_date=True,)

# Sort data by year # Syntax: {} = dictionary; [] = list; () = arguments for functions
df_all = df_all.sort_index() 

# Generate share of net exports in GDP
net_exports_share = df_all['Exports (% of GDP)'] - df_all['Imports (% of GDP)']

# Insert 'net_exports_share' as the 6th column, labeled 'Net exports (% of GDP)'
df_all.insert(6, 'Net exports (% of GDP)', net_exports_share)

# Generate net exports annual growth
net_exports_growth = (df_all['Exports (current prices, USD)'] - df_all['Imports (current prices, USD)']).pct_change() * 100

# Insert 'net_exports_growth' as the 12th column, labeled 'Net exports (annual growth, %)'
df_all.insert(12, 'Net exports (annual growth, %)', net_exports_growth)

# Generate contributions to GDP growth using a loop
components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
              'Exports', 'Imports', 'Agriculture', 'Industry (including construction)', 'Services']
for item in components:
    df_all[f'{item} (contribution, pp)'] = df_all[f'{item} (% of GDP)'].shift(1) / 100 * df_all[f'{item} (annual growth, %)']

# Generate net export contribution to GDP growth
df_all['Net exports (contribution, pp)'] = df_all['Exports (contribution, pp)'] - df_all['Imports (contribution, pp)']

# Round to one decimal
df_all = df_all.round(1)

# Create dataframe for top three sources of growth on the demand/production side
demand_components = ['Private consumption', 'Government expenditure', 'Gross capital formation', 
                     'Net exports']
production_components = ['Agriculture', 'Industry (including construction)', 'Services']
components = demand_components + production_components

# df_all # Python returns output of the last command, so this won't be displayed if the rest is not commented out

# Create dataframe for top sources of growth on the demand side ## QUESTION: How does the 'in-line' loop work? >>> Thru list comprehensions
top_demand_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_demand_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_demand_contributions.index]

# Create dataframe for top shares of GDP on the demand side
top_demand_shares = df_all.last('Y')[[f'{item} (% of GDP)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_demand_shares.index = [index.replace(' (% of GDP)', '') for index in top_demand_shares.index]

# Create dataframe for top growth rates of GDP components on the demand side
top_demand_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in demand_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_demand_growth.index = [index.replace(' (annual growth, %)', '') for index in top_demand_growth.index]

# Create dataframe for top sources of growth on the production side >>>> Loop across 'demand' + 'production' >>> Possible but impractical since we need to refer to separate dataframes in the writeup
top_production_contributions = df_all.last('Y')[[f'{item} (contribution, pp)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0],ascending=False)
top_production_contributions.index = [index.replace(' (contribution, pp)', '') for index in top_production_contributions.index]

# Create dataframe for top shares of GDP on the supply side
top_production_shares = df_all.last('Y')[[f'{item} (% of GDP)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_production_shares.index = [index.replace(' (% of GDP)', '') for index in top_production_shares.index]

# Create dataframe for top growth rates of GDP components on the production side
top_production_growth = df_all.last('Y')[[f'{item} (annual growth, %)' for item in production_components]].transpose().sort_values(by=df_all.last('Y').index[0], ascending=False)
top_production_growth.index = [index.replace(' (annual growth, %)', '') for index in top_production_growth.index]

In [6]:
#Create blank .docx where the output will be exported
doc = docx.Document()

# Add title (heading level 0)
doc.add_heading(f'{country_codes.country_name[country]}: Recent Economic Developments and Outlook', level=0) ## QUESTION: Why '[country]'?

# df_all

# Set placeholders (.last('2Y') to allow comparison btween values, wout/ it, Python would try and compare objects (and fail))
change = ''
if df_all['GDP growth (%)'].last('2Y').values[-1] > df_all['GDP growth (%)'].last('2Y').values[-2]:
    change = random.choice(['increased', 'picked up', 'rose'])
elif df_all['GDP growth (%)'].last('2Y').values[-1] == df_all['GDP growth (%)'].last('2Y').values[-2]:
    change = random.choice(['remained at', 'stayed at'])
else:
    change = random.choice(['decreased', 'contracted', 'slowed down'])

# Add a heading for GDP growth section
doc.add_heading(f'GDP growth in {dt.date.today().year - 1} {change}', level=1)


# Add paragraph on GDP growth
# QUESTIONS:    Display line numbers? Select the cell and click L.
#               What to do when not running & error appears elsewhere (comtrade)? > Read error message
#               Git fetch / commit / push / make pull request

# TODO: for demand-side contributions: 
#                   1 - Contributions in the same direction as overall GDP growth:
#                       AA contributed to the most (X pp)
#                       [followed by BB (X pp) (if applicable)]
#                       [,/and] CC (X pp) (if applicable)]
#                       [and] DD (X pp) (if applicable)].
#                   2 - Contributions in the opposite direction:                  
#                       [On the other hand, EE contributed negatively/positively (X pp) (if applicable)] 
#                       [, as well as FF (X pp) (if applicable)],
#                       [GG (X pp) (if applicable)]
#                       [and HH (X pp) (if applicable)]].
#               Adjust '({top_demand_contributions.values[0]}% of GDP) \' to show share of top contributor

#top_demand_contributions.index.index[0].lower()

p = doc.add_paragraph(f"GDP growth {change} from {df_all['GDP growth (%)'].last('2Y').values[-2]}% in {df_all['GDP growth (%)'].last('2Y').index[-2].year} to {df_all['GDP growth (%)'].last('2Y').values[-1]}% in {df_all['GDP growth (%)'].last('2Y').index[-1].year}. On the demand side, {top_demand_contributions.index[0].lower()}, accounting for {top_demand_shares.loc[top_demand_contributions.index[0]].values[0]}% of GDP, contributed the most with {top_demand_contributions.values.max()} percentage points (pp).")

i = 1
while top_demand_contributions.values[i][0] > 0:
    p.add_run(f" {top_demand_contributions.index[i]} {random.choice(['added', 'gave', 'shared',])} {top_demand_contributions.values[i][0]}pp.")
    i += 1
    if i >= len(top_demand_contributions):
        break 

if sum(top_demand_contributions.values < 0)[0] == 1:
    p.add_run(f" On the other hand, {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-1].lower()} {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_demand_contributions.values[-1][0])}pp from growth.")
elif sum(top_demand_contributions.values < 0)[0] == 2:
    p.add_run(f" On the other hand, {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-1].lower()} and {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-2].lower()} {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_demand_contributions.values[-1][0])}pp and {abs(top_demand_contributions.values[-2][0])}pp from growth, respectively.")
elif sum(top_demand_contributions.values < 0)[0] == 3:
    p.add_run(f" On the other hand, {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-1].lower()}, {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-2].lower()}, as well as  {top_demand_contributions.loc[top_demand_contributions.values < 0].index[-3].lower()}, {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_demand_contributions.values[-1][0])}pp, {abs(top_demand_contributions.values[-2][0])}pp, and {abs(top_demand_contributions.values[-3][0])}pp from growth, respectively.")

if sum(top_demand_contributions.values == 0)[0] > 0:
    p.add_run(f" Meanwhile, {top_demand_contributions.loc[top_demand_contributions.values == 0].index[-1].lower()} had trivial contribution to growth this period.")

p.add_run(f" On the supply side, {top_production_contributions.index[0].lower()}, accounting for {top_production_shares.loc[top_production_contributions.index[0]].values[0]}% of GDP, contributed the most with {top_production_contributions.values.max()}pp.")

i = 1
while top_production_contributions.values[i][0] > 0:
    p.add_run(f" {top_production_contributions.index[i]} {random.choice(['added', 'gave', 'shared',])} {top_production_contributions.values[i][0]}pp.")
    i += 1
    if i >= len(top_production_contributions):
        break 

if sum(top_production_contributions.values < 0)[0] == 1:
    p.add_run(f" However, {top_production_contributions.loc[top_production_contributions.values < 0].index[-1].lower()} {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_production_contributions.values[-1][0])}pp from growth.")
elif sum(top_production_contributions.values < 0)[0] == 2:
    p.add_run(f" However, {top_production_contributions.loc[top_production_contributions.values < 0].index[-1].lower()} and {top_production_contributions.loc[top_production_contributions.values < 0].index[-2].lower()} {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_production_contributions.values[-1][0])}pp and {abs(top_production_contributions.values[-2][0])}pp from growth, respectively.")
elif sum(top_production_contributions.values < 0)[0] == 3:
    p.add_run(f" However, {top_production_contributions.loc[top_production_contributions.values < 0].index[-1].lower()}, {top_production_contributions.loc[top_production_contributions.values < 0].index[-2].lower()}, as well as  {top_production_contributions.loc[top_production_contributions.values < 0].index[-3].lower()}, {random.choice(['subtracted', 'cut', 'shaved',])} {abs(top_production_contributions.values[-1][0])}pp, {abs(top_production_contributions.values[-2][0])}pp, and {abs(top_production_contributions.values[-3][0])}pp from growth, respectively.")

if sum(top_production_contributions.values == 0)[0] == 1:
    p.add_run(f" Meanwhile, {top_production_contributions.loc[top_production_contributions.values == 0].index[-1].lower()} had trivial contribution to growth this period.")
elif sum(top_production_contributions.values == 0)[0] == 2:
    p.add_run(f" Meanwhile, {top_production_contributions.loc[top_production_contributions.values == 0].index[-1].lower()} and {top_production_contributions.loc[top_production_contributions.values == 0].index[-2].lower()} had trivial contribution to growth this period.")


# Justify paragraph 'p'
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [7]:
# Create the dataframe that will be plotted
df_chart_gdp = df_all[['GDP growth (%)'] + [f'{item} (contribution, pp)' for item in demand_components + production_components]].dropna()
df_chart_gdp.index = df_chart_gdp.index.year
df_chart_gdp.reset_index(inplace=True)

# Create a Pandas Excel writer using xlsxwriter as the engine
writer = pd.ExcelWriter(f'{country}-charts.xlsx', engine='xlsxwriter')
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in demand_components]].to_excel(writer, sheet_name='GDP-demand', index=False)
df_chart_gdp[['date', 'GDP growth (%)'] + [f'{item} (contribution, pp)' for item in production_components]].to_excel(writer, sheet_name='GDP-production', index=False)

# Create a blank canvas
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
plt.style.use('default')

# Create a stacked bar chart of the components
df_chart_gdp[[f'{item} (contribution, pp)' for item in demand_components]].plot(kind='bar', stacked=True, ax=axs[0])

df_chart_gdp[[f'{item} (contribution, pp)' for item in production_components]].plot(kind='bar', stacked=True, ax=axs[1])

# Create a line plot of the GDP growth series
for i in range(len(axs)):
    axs[i].plot(df_chart_gdp.loc[:, 'GDP growth (%)'], lw=2.5, marker='D', markersize=10, color='black')
    # Set labels in the x-axis
    axs[i].set_xticklabels(df_chart_gdp.date, rotation=0)
    axs[i].set_ylabel('percentage points')
    axs[i].yaxis.set_major_locator(ticker.MaxNLocator(6))
    axs[i].set_xlabel('')

# Set legend
axs[0].legend(['GDP growth (%)'] + demand_components)
axs[1].legend(['GDP growth (%)'] + production_components)

# Set graph formatting and save to local folder
axs[0].set_title('Demand-side contributions to growth')
axs[1].set_title('Supply-side contributions to growth')

plt.tight_layout()
plt.savefig(f'{country}_GDP_components.png')
plt.close()

# Add picture to the document
graph = doc.add_picture(f'{country}_GDP_components.png', width=Cm(14), height=Cm(7))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [8]:
# Add a heading for Expenditure Approach section
doc.add_heading(f'{top_demand_growth.index[0]} achieved largest gain on the demand side', level=2)

# Write paragraphs for each demand component using a loop
change = {}
for item in demand_components:
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > 0:
        change[f'{item}'] = random.choice(['expanded', 'grew', 'increased', 'picked up', 'jumped'])
    else:
        change[f'{item}'] = random.choice(['decreased', 'contracted', 'shrank', 'declined', 'plunged'])

p = doc.add_paragraph(f"{top_demand_growth.index[0]} {change[top_demand_growth.index[0]]} by the biggest margin at {top_demand_growth.values.max()}% annual growth.")

i = 1
while top_demand_growth.values[i][0] > 0:
    p.add_run(f" {top_demand_growth.index[i]} {change[top_demand_growth.index[i]]} by {top_demand_growth.values[i][0]}%.") 
    i += 1
    if i >= len(top_demand_growth):
        break 

for i in range(1, len(top_demand_growth)):
    if top_demand_growth.values[i][0] < 0:
        p.add_run(f" On the other hand, {top_demand_growth.index[i].lower()} {change[top_demand_growth.index[i]]} by {abs(top_demand_growth.values[i][0])}%.")
    elif top_demand_growth.values[i][0] == 0:
        p.add_run(f" Meanwhile, {top_demand_growth.index[i].lower()} kept unchanged this period.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Add a heading for Production Approach section
doc.add_heading(f"On the supply side, growth in {top_production_growth.index[0].lower()} accelerated the fastest", level=2)


change = {}
for item in production_components:
    if df_all[f'{item} (annual growth, %)'].last('2Y').values[-1] > 0:
        change[f'{item}'] = random.choice(['expanded', 'grew', 'increased', 'picked up', 'jumped'])
    else:
        change[f'{item}'] = random.choice(['decreased', 'contracted', 'shrank', 'declined', 'plunged'])

p = doc.add_paragraph(f"{top_production_growth.index[0]} {change[top_production_growth.index[0]]} by the largest edge at {top_production_growth.values.max()}% annual growth.")

i = 1
while top_production_growth.values[i][0] > 0:
    p.add_run(f" {top_production_growth.index[i]} {change[top_production_growth.index[i]]} by {top_production_growth.values[i][0]}%.") 
    i += 1
    if i >= len(top_production_growth):
        break 

for i in range(1, len(top_production_growth)):
    if top_production_growth.values[i][0] < 0:
        p.add_run(f" On the other hand, {top_production_growth.index[i].lower()} {change[top_production_growth.index[i]]} by {abs(top_production_growth.values[i][0])}%.")
    elif top_production_growth.values[i][0] == 0:
        p.add_run(f" Meanwhile, {top_production_growth.index[i].lower()} kept unchanged this period.")

p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [9]:
# Write section on income approach

# Set placeholders
phillips = ['Unemployment rate (%)', 'Inflation rate (%)']
change = {}
for item in phillips:
    if df_all[item].last('2Y').values[-1] > df_all[item].last('2Y').values[-2]:
        change[item] = random.choice(['worsened', 'increased', 'jumped'])
    elif df_all[item].last('2Y').values[-1] == df_all[item].last('2Y').values[-2]:
        change[item] = random.choice(['remained', 'stayed'])
    else:
        change[item] = random.choice(['improved', 'declined', 'plunged'])

# Add a heading for the income approach section
doc.add_heading(f"Unemployment {change['Unemployment rate (%)']}; inflation {change['Inflation rate (%)']}", level=2)

# Add a paragraph on unemployment rate
p = doc.add_paragraph(f"Unemployment rate {change['Unemployment rate (%)']} from {df_all['Unemployment rate (%)'].last('2Y').values[-2]}% in {df_all['Unemployment rate (%)'].last('2Y').index[-2].year} to {df_all['Unemployment rate (%)'].last('2Y').values[-1]}% in {df_all['Unemployment rate (%)'].last('2Y').index[-1].year}. Consequently, inflation {change['Inflation rate (%)']} from {'a deflation of' + str(abs(df_all['Inflation rate (%)'].last('2Y').values[-2])) if df_all['Inflation rate (%)'].last('2Y').values[-2] < 0 else df_all['Inflation rate (%)'].last('2Y').values[-2]}% to {'a deflation of' + str(abs(df_all['Inflation rate (%)'].last('2Y').values[-1])) if df_all['Inflation rate (%)'].last('2Y').values[-1] < 0 else df_all['Inflation rate (%)'].last('2Y').values[-1]}%. {'At the end of the year, the central bank set the policy rate at ' + str(df_policyrate_annual.last('Y').values[0][0]) + '%' if type(df_policyrate_annual) is not dict else ''}.")
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [10]:
# Create the dataframe that will be plotted
df_chart_phillips = pd.DataFrame(df_all[['Unemployment rate (%)', 'Inflation rate (%)']].last('5Y'))
df_chart_phillips.index = df_chart_phillips.index.year
df_chart_phillips.reset_index(inplace=True)

# Create a Pandas Excel writer using xlsxwriter as the engine
for item in phillips:
    df_chart_phillips[['date', f'{item}']].to_excel(writer, sheet_name=f'{item}', index=False)

# Close the Pandas Excel writer and output the Excel file
writer.save()

# Create a blank canvas
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
plt.style.use('default')

# Create a bar chart of inflation and CAB
df_chart_phillips['Unemployment rate (%)'].plot(kind='bar', ax=axs[0])
df_chart_phillips['Inflation rate (%)'].plot(kind='bar', ax=axs[1])

# Set graph formatting
for i in range(len(axs)):
    axs[i].set_xticklabels(df_chart_phillips.date, rotation=0)
    axs[i].set_xlabel('')
    axs[i].axes.get_yaxis().set_visible(False)

axs[0].set_title('Unemployment rate (%)')
axs[1].set_title('Inflation rate (%)')

# Set data labels for unemployment
labels_unem = df_chart_phillips['Unemployment rate (%)']
rects_unem = axs[0].patches

for rect, label in zip(rects_unem, labels_unem):
    height = rect.get_height()
    if height > 0:
        axs[0].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%', ha='center', va='bottom')
    else:
        axs[0].text(rect.get_x() + rect.get_width() / 2, height - 0.1, str(label) + '%', ha='center', va='bottom')

# Set data labels for inflation
labels_inf = df_chart_phillips['Inflation rate (%)']
rects_inf = axs[1].patches

for rect, label in zip(rects_inf, labels_inf):
    height = rect.get_height()
    if height > 0:
        axs[1].text(rect.get_x() + rect.get_width() / 2, height, str(label) + '%', ha='center', va='bottom')
    else:
        axs[1].text(rect.get_x() + rect.get_width() / 2, height  - 0.2, str(label) + '%', ha='center', va='bottom')
    
# Save to local folder
plt.savefig(f'{country}_unemployment_inflation.png')
plt.close()

# Add picture to the document
graph = doc.add_picture(f'{country}_unemployment_inflation.png', width=Cm(14), height=Cm(7))
last_paragraph = doc.paragraphs[-1]
last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [11]:
# Set placeholders
sign_latest = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-1] > 0 else 'deficit'
sign_old = 'surplus' if df_all['Current account balance (% of GDP)'].last('2Y').values[-2] > 0 else 'deficit'

# Add a heading for the CAB analysis section
doc.add_heading(f'Current account balance posted a {sign_latest}', level=2)

# Add a paragraph on unemployment rate
p = doc.add_paragraph(f"Current account balance (CAB) recorded a {sign_latest} at {abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-1])}% of GDP in {df_all['Current account balance (% of GDP)'].last('2Y').index[-1].year}. Net trade in goods and services reached USD {round(df_all['Net trade in goods and services (current USD)'].last('2Y').values[-1] / 1e9, 1)} billion. In {df_all['Current account balance (% of GDP)'].last('2Y').index[-2].year}, CAB posted a {sign_latest} at {abs(df_all['Current account balance (% of GDP)'].last('2Y').values[-2])}% of GDP.")
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Needs to be changed for each machine/day on which the code is run
# Recover from url by downloading any .csv here: https://comtrade.un.org/data/
uitoken = 'b3f9aeb66ba24558248a2c38acebc1f'

# Download export data
# lambda = create a function, with (in this case) x as the single argument
# ['Partner', 'share']] > Accessing Partner & share columns of the DF, [] access list of columns
url_destination = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=2&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_destination = pd.read_csv(url_destination)
mask1 = (df_destination['Partner']=='World') & (df_destination['Mode of Transport']=='All MOTs') & (df_destination['2nd Partner']=='World')
total_exports = df_destination[mask1]['Trade Value (US$)'].values[0]
mask2 = (~(df_destination['Partner'] == 'World')) & (df_destination['Mode of Transport'] == 'All MOTs') & (df_destination['2nd Partner']=='World')
df_destination = df_destination[mask2]
df_destination = df_destination.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_destination = df_destination.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)

# Download import data
url_origin = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=1&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_origin = pd.read_csv(url_origin)
mask1 = (df_origin['Partner']=='World') & (df_origin['Mode of Transport']=='All MOTs') & (df_origin['2nd Partner']=='World')
total_imports = df_origin[mask1]['Trade Value (US$)'].values[0]
mask2 = (~(df_origin['Partner'] == 'World')) & (df_origin['Mode of Transport'] == 'All MOTs') & (df_origin['2nd Partner']=='World')
df_origin = df_origin[mask2]
df_origin = df_origin.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_origin = df_origin.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)

url_exports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=2&cc=AG2&uitoken={uitoken}&fmt=csv"
df_exports = pd.read_csv(url_exports)
mask = (df_exports['Mode of Transport']=='All MOTs') & (df_exports['2nd Partner']=='World')
df_exports = df_exports[mask]
df_exports = df_exports.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_exports = df_exports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)

url_imports = f"https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=1&cc=AG2&uitoken={uitoken}&fmt=csv"
df_imports = pd.read_csv(url_imports)
mask = (df_imports['Mode of Transport']=='All MOTs') & (df_imports['2nd Partner']=='World')
df_imports = df_imports[mask]
df_imports = df_imports.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_imports = df_imports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)

p = doc.add_paragraph(f"{top_destination['Partner'].values[0]} is the country's top export destination accounting for {top_destination['share'].values[0]}% of total exports in {dt.date.today().year - 1}. Other major exports partners include {top_destination['Partner'].values[1]} ({top_destination['share'].values[1]}%), {top_destination['Partner'].values[2]} ({top_destination['share'].values[2]}%), {top_destination['Partner'].values[3]} ({top_destination['share'].values[3]}%), and {top_destination['Partner'].values[4]} ({top_destination['share'].values[4]}%). Top export commodities are {top_exports['Commodity'].values[0].lower()}, accounting for {top_exports['share'].values[0]}% of total exports.")
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# p = doc.add_paragraph(f"Top export commodities include {top_exports['Commodity'].values[0].lower()} ({top_exports['share'].values[0]}% of total exports), {top_exports['Commodity'].values[1].lower()} ({top_exports['share'].values[1]}%), {top_exports['Commodity'].values[2].lower()} ({top_exports['share'].values[2]}%), {top_exports['Commodity'].values[3].lower()} ({top_exports['share'].values[3]}%), and {top_exports['Commodity'].values[4].lower()} ({top_exports['share'].values[4]}%).")
# p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

p = doc.add_paragraph(f"For imports, top imports origin in {dt.date.today().year - 1} is {top_origin['Partner'].values[0]} ({top_origin['share'].values[0]}% of total imports), followed by {top_origin['Partner'].values[1]} ({top_origin['share'].values[1]}%), {top_origin['Partner'].values[2]} ({top_origin['share'].values[2]}%), {top_origin['Partner'].values[3]} ({top_origin['share'].values[3]}%),  and {top_origin['Partner'].values[4]} ({top_origin['share'].values[4]}%). Major import commodities are {top_imports['Commodity'].values[0].lower()}, accounting for {top_imports['share'].values[0]}% of total imports.")
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# p = doc.add_paragraph(f"Major import commodities include {top_imports['Commodity'].values[0].lower()} ({top_imports['share'].values[0]}% of total imports), {top_imports['Commodity'].values[1].lower()} ({top_imports['share'].values[1]}%), {top_imports['Commodity'].values[2].lower()} ({top_imports['share'].values[2]}%), {top_imports['Commodity'].values[3].lower()} ({top_imports['share'].values[3]}%),  and {top_imports['Commodity'].values[4].lower()} ({top_imports['share'].values[4]}%).")
# p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [12]:
if type(df_gdp_growth_quarterly) is not dict:
    change = {}
    for item in df_gdp_growth_quarterly.columns:
        if df_gdp_growth_quarterly[item].last('2Q').values[-1] > 0:
            change[f'{item}'] = random.choice(['expanded', 'grew', 'increased', 'picked up', 'jumped'])
        else:
            change[f'{item}'] = random.choice(['decreased', 'contracted', 'shrank', 'declined', 'plunged'])
    
    # Create dataframe for top growth rates of GDP components on the demand side
    top_demand_growth_quarterly = df_gdp_growth_quarterly.last('Q')[[item for item in df_gdp_growth_quarterly.columns if item != 'GDP growth']].transpose().sort_values(by=df_gdp_growth_quarterly.last('Q').index[0], ascending=False)

    p = doc.add_heading(f"Output {'contracted' if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1] < 0 else 'expanded'} in Q{df_gdp_growth_quarterly.last('2Q').index[-1].quarter} {df_gdp_growth_quarterly.last('2Q').index[-1].year}", level=1)
    p = doc.add_paragraph(f"Output {'plunged' if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1] < 0 else 'jumped'} by {abs(df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1])}% year-on-year in Q{df_gdp_growth_quarterly.last('2Q').index[-1].quarter} of {df_gdp_growth_quarterly.last('2Q').index[-1].year}. Growth in overall economic activity {'improved' if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1] > df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2] else 'worsened'} from {'a contraction of ' + str(abs(df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2])) if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2] < 0 else df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2]}% in the previous quarter. {top_demand_growth_quarterly.index[0]} {change[top_demand_growth_quarterly.index[0]]} by the biggest margin at {top_demand_growth_quarterly.values.max()}% annual growth.")

    i = 1
    while top_demand_growth_quarterly.values[i][0] > 0:
        p.add_run(f" {top_demand_growth_quarterly.index[i]} {change[top_demand_growth_quarterly.index[i]]} by {top_demand_growth_quarterly.values[i][0]}%.") 
        i += 1
        if i >= len(top_demand_growth_quarterly):
            break 
    
    if sum(top_demand_growth_quarterly.values < 0)[0] == 1:
        p.add_run(f" On the other hand, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-1].lower()} {change[top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-1]]} by {abs(top_demand_growth_quarterly.values[-1][0])}%.")
    elif sum(top_demand_growth_quarterly.values < 0)[0] == 2:
        p.add_run(f" On the other hand, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-1].lower()} and {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-2].lower()} {change[top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-2]]} by {abs(top_demand_growth_quarterly.values[-1][0])}% and {abs(top_demand_growth_quarterly.values[-2][0])}%, respectively.")
    elif sum(top_demand_growth_quarterly.values < 0)[0] == 3:
        p.add_run(f" On the other hand, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-1].lower()}, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-2].lower()}, as well as  {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-3].lower()}, {change[top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-3]]} by {abs(top_demand_growth_quarterly.values[-1][0])}%, {abs(top_demand_growth_quarterly.values[-2][0])}%, and {abs(top_demand_growth_quarterly.values[-3][0])}%, respectively.")
    else:
        p.add_run(f" On the other hand, the rest of the components, namely {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-1].lower()}, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-2].lower()}, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-3].lower()}, as well as  {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-4].lower()}, {change[top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values < 0].index[-4]]} by {abs(top_demand_growth_quarterly.values[-1][0])}%, {abs(top_demand_growth_quarterly.values[-2][0])}%, {abs(top_demand_growth_quarterly.values[-3][0])}%, and {abs(top_demand_growth_quarterly.values[-4][0])}%, respectively.")


    if sum(top_demand_growth_quarterly.values == 0)[0] > 0:
        p.add_run(f" Meanwhile, {top_demand_growth_quarterly.loc[top_demand_growth_quarterly.values == 0].index[-1].lower()} was flat over the same period.")


    # change = {}
    # for item in df_gdp_growth_quarterly.columns:
    #     # Set placeholders
    #     if df_gdp_growth_quarterly[f'{item}'].last('2Q').values[-1] > df_gdp_growth_quarterly[f'{item}'].last('2Q').values[-2]:
    #         change[f'{item}'] = random.choice(['improved', 'increased', 'jumped'])
    #     elif df_gdp_growth_quarterly[f'{item}'].last('2Q').values[-1] == df_gdp_growth_quarterly[f'{item}'].last('2Q').values[-2]:
    #         change[f'{item}'] = random.choice(['kept still', 'stayed flat'])
    #     else:
    #         change[f'{item}'] = random.choice(['worsened', 'declined', 'fell'])
    # p = doc.add_heading(f"Output {'contracted' if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1] < 0 else 'expanded'} in Q{df_gdp_growth_quarterly.last('2Q').index[-1].quarter} {df_gdp_growth_quarterly.last('2Q').index[-1].year}", level=1)
    # p = doc.add_paragraph(f"Output {'plunged' if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1] < 0 else 'jumped'} by {abs(df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-1])}% year-on-year in Q{df_gdp_growth_quarterly.last('2Q').index[-1].quarter} of {df_gdp_growth_quarterly.last('2Q').index[-1].year}. Growth in overall economic activity {change['GDP growth']} from {'a contraction of ' + str(abs(df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2])) if df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2] < 0 else df_gdp_growth_quarterly['GDP growth'].last('2Q').values[-2]}% in the previous quarter.")
    # for item in demand_components:
    #     if item in df_gdp_growth_quarterly.columns:
    #         p.add_run(f" {item} growth {change[item]} from {df_gdp_growth_quarterly[item].last('2Q').values[-2]}% to {df_gdp_growth_quarterly[item].last('2Q').values[-1]}%.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [13]:
if (type(df_retail) is not dict) or (df_retail['codelists']['codesfound']):
    change = {}
    for item in df_retail.columns:
        # Set placeholders
        if df_retail[f'{item}'].values[-1] > df_retail[f'{item}'].values[-2]:
            change[f'{item}'] = random.choice(['improved', 'increased', 'jumped'])
        elif df_retail[f'{item}'].values[-1] == df_retail[f'{item}'].values[-2]:
            change[f'{item}'] = random.choice(['remained at', 'stayed at'])
        else:
            change[f'{item}'] = random.choice(['worsened', 'declined', 'contracted'])
    p = doc.add_heading(f"Retail sales {'contracted' if df_retail['Retail sales index'].values[-1] < 0 else 'picked up'}", level=2)
    p = doc.add_paragraph(f"Retail sales {'contracted' if df_retail['Retail sales index'].values[-1] < 0 else 'picked up'} by {abs(df_retail['Retail sales index'].values[-1])}% year-on-year in {calendar.month_name[df_retail.index[-1].month]} of {df_retail.index[-1].year}. Growth in the retail sector {change['Retail sales index']} from a{' contraction of ' + str(abs(df_retail[f'Retail sales index'].values[-2])) + '%' if df_retail[f'Retail sales index'].values[-2] < 0 else 'n expansion of ' +  str(df_retail[f'Retail sales index'].values[-2])} in {calendar.month_name[df_retail.index[-1].month - 1]}, reflecting {'increased' if df_retail[f'{item}'].values[-1] > df_retail[f'{item}'].values[-2] else 'muted'} trade activity.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

if type(df_consumer) is not dict:
    change = {}
    for item in df_consumer.columns:
        # Set placeholders
        if df_consumer[f'{item}'].last('2Q').values[-1] > df_consumer[f'{item}'].last('2Q').values[-2]:
            change[f'{item}'] = random.choice(['improved', 'increased', 'jumped'])
        elif df_consumer[f'{item}'].last('2Q').values[-1] == df_consumer[f'{item}'].last('2Q').values[-2]:
            change[f'{item}'] = random.choice(['remained at', 'stayed at'])
        else:
            change[f'{item}'] = random.choice(['worsened', 'declined', 'contracted'])
    p = doc.add_heading(f"Consumer confidence {'up' if df_consumer[f'Consumer confidence index'].last('2Q').values[-1] > 0 else 'down'}", level=2)
    p = doc.add_paragraph(f"Meanwhile, consumer confidence index was in the {'positive' if df_consumer[f'Consumer confidence index'].last('2Q').values[-1] > 0 else 'negative'} territory at {df_consumer[f'Consumer confidence index'].last('2Q').values[-1]} points in Q{df_consumer.last('2Q').index[-1].quarter} of {df_consumer.last('2Q').index[-1].year}. Confidence {change['Consumer confidence index']} from {df_consumer[f'Consumer confidence index'].last('2Q').values[-2]} points in the previous quarter. Expectations of consumers about the general economic situation in the next 12 months turned {'pessimistic' if df_consumer[f'Consumer expectations index'].last('2Q').values[-1] < 0 else 'optimistic'} at {df_consumer[f'Consumer expectations index'].last('2Q').values[-1]} points from {df_consumer[f'Consumer expectations index'].last('2Q').values[-2]} points over the same period, reflecting {'worsened' if df_consumer[f'Consumer expectations index'].last('2Q').values[-1] < df_consumer[f'Consumer expectations index'].last('2Q').values[-2] else 'improved'} consumer sentiments.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [14]:
if (type(df_production) is not dict) or (df_production['codelists']['codesfound']):
# Set placeholders
    change_production = {}
    for item in df_production.columns:
        if df_production[f'{item}'].values[-1] > df_production[f'{item}'].values[-2]:
            change_production[f'{item}'] = random.choice(['improved', 'increased', 'jumped'])
        elif df_production[f'{item}'].values[-1] == df_production[f'{item}'].values[-2]:
            change_production[f'{item}'] = random.choice(['remained at', 'stayed at'])
        else:
            change_production[f'{item}'] = random.choice(['worsened', 'declined', 'contracted'])
    # Add a paragraph on Production indices
    if all(item in df_production.columns for item in ['Industrial production', 'Manufacturing',
                                                      'Mining and quarrying', 
                                                      'Water supply, sewerage, waste management & remediation',
                                                      'Electricity, gas, steam and air conditioning supply']):
        p = doc.add_heading(f"Industrial output {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'}", level=2)
        p = doc.add_paragraph(f"Industrial production {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'} by {abs(df_production['Industrial production'].values[-1])}% year-on-year in {calendar.month_name[df_production.index[-1].month]}, a{'n increase' if df_production['Industrial production'].values[-1] > df_production['Industrial production'].values[-2] else ' decrease'} from {df_production['Industrial production'].values[-2]}% growth in the previous month. Looking at the details, growth in manufacturing {change_production['Manufacturing']} to {df_production['Manufacturing'].values[-1]}% from {df_production['Manufacturing'].values[-2]}%, while mining and quarrying output growth {change_production['Mining and quarrying']} to {df_production['Mining and quarrying'].values[-1]}% from {df_production['Mining and quarrying'].values[-2]}%. Meanwhile, growth in water supply, sewerage, waste management & remediation {change_production['Water supply, sewerage, waste management & remediation']} to {df_production['Water supply, sewerage, waste management & remediation'].values[-1]}% from {df_production['Water supply, sewerage, waste management & remediation'].values[-2]}%, while electricity, gas, steam and air conditioning supply output growth {change_production['Electricity, gas, steam and air conditioning supply']} to {df_production['Electricity, gas, steam and air conditioning supply'].values[-1]}% from {df_production['Electricity, gas, steam and air conditioning supply'].values[-2]}%")
    elif all(item in df_production.columns for item in ['Industrial production', 'Manufacturing']):
        p = doc.add_heading(f"Industrial output {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'}", level=2)
        p = doc.add_paragraph(f"Industrial production {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'} by {abs(df_production['Industrial production'].values[-1])}% year-on-year in {calendar.month_name[df_production.index[-1].month]}, a{'n increase' if df_production['Industrial production'].values[-1] > df_production['Industrial production'].values[-2] else 'decrease'} from {df_production['Industrial production'].values[-2]}% growth in the previous month. Growth in manufacturing {change_production['Manufacturing']} to {df_production['Manufacturing'].values[-1]}% from {df_production['Manufacturing'].values[-2]}%.")
    elif 'Industrial production' in df_production.columns:
        p = doc.add_heading(f"Industrial output {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'}", level=2)
        p = doc.add_paragraph(f"Industrial production {'shrank' if df_production['Industrial production'].values[-1] < 0 else 'expanded'} by {abs(df_production['Industrial production'].values[-1])}% year-on-year in {calendar.month_name[df_production.index[-1].month]}, a{'n increase' if df_production['Industrial production'].values[-1] > df_production['Industrial production'].values[-2] else 'decrease'} from {df_production['Industrial production'].values[-2]}% growth in the previous month.")
    elif 'Manufacturing' in df_production.columns:
        p = doc.add_heading(f"Industrial output {'shrank' if df_production['Manufacturing'].values[-1] < 0 else 'expanded'}", level=2)
        p = doc.add_paragraph(f"Manufacturing {'shrank' if df_production['Manufacturing'].values[-1] < 0 else 'expanded'} by {abs(df_production['Manufacturing'].values[-1])}% year-on-year in {calendar.month_name[df_production.index[-1].month]}, a{'n increase' if df_production['Manufacturing'].values[-1] > df_production['Manufacturing'].values[-2] else ' decrease'} from {df_production['Manufacturing'].values[-2]}% growth in the previous month.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [15]:
if (type(df_cpi) is not dict) or (df_cpi['codelists']['codesfound']):
    # Set placeholders
    change_cpi = {}
    for item in df_cpi.columns:
        if df_cpi[f'{item}'].values[-1] > df_cpi[f'{item}'].values[-2]:
            change_cpi[f'{item}'] = random.choice(['worsened', 'rose', 'jumped'])
        elif df_cpi[f'{item}'].values[-1] == df_cpi[f'{item}'].values[-2]:
            change_cpi[f'{item}'] = random.choice(['remained at', 'stayed at'])
        else:
            change_cpi[f'{item}'] = random.choice(['improved', 'declined', 'slowed down'])
    # Add a paragraph on Production indices
    if all(item in df_cpi.columns for item in ['Overall inflation', 'Food', 'Housing, rent, water, electricity, gas & other fuels', 'Transportation', 'Communication', 'Health/medical care', 'Recreation', 'Education']):
        p = doc.add_heading(f"Inflation {change_cpi['Overall inflation']}", level=2)
        p = doc.add_paragraph(f"Overall inflation {change_cpi['Overall inflation']} to {df_cpi['Overall inflation'].values[-1]}% year-on-year in {calendar.month_name[df_cpi.index[-1].month]} from {df_cpi['Overall inflation'].values[-2]}% in the previous month. Prices for food products {change_cpi['Food']} to {df_cpi['Food'].values[-1]}% from {df_cpi['Food'].values[-2]}%, while housing, rent, water, electricity, gas & other fuels {change_cpi['Housing, rent, water, electricity, gas & other fuels']} to {df_cpi['Housing, rent, water, electricity, gas & other fuels'].values[-1]}% from {df_cpi['Housing, rent, water, electricity, gas & other fuels'].values[-2]}%. Transportation {change_cpi['Transportation']} to {df_cpi['Transportation'].values[-1]}% from {df_cpi['Transportation'].values[-2]}%, while communication {change_cpi['Communication']} to {df_cpi['Communication'].values[-1]}% from {df_cpi['Communication'].values[-2]}%. Meanwhile, prices for health/medical care {change_cpi['Health/medical care']} to \
{df_cpi['Health/medical care'].values[-1]}% from {df_cpi['Health/medical care'].values[-2]}%, recreation {change_cpi['Recreation']} to {df_cpi['Recreation'].values[-1]}% from {df_cpi['Recreation'].values[-2]}%, and education {change_cpi['Education']} to {df_cpi['Education'].values[-1]}% from {df_cpi['Education'].values[-2]}%")
    elif all(item in df_cpi.columns for item in ['Overall inflation', 'Food']):
        p = doc.add_heading(f"Inflation {change_cpi['Overall inflation']}", level=2)
        p = doc.add_paragraph(f"Overall inflation {change_cpi['Overall inflation']} to {df_cpi['Overall inflation'].values[-1]}% year-on-year in {calendar.month_name[df_cpi.index[-1].month]} from {df_cpi['Overall inflation'].values[-2]}% in the previous month. Prices for food products {change_cpi['Food']} to {df_cpi['Food'].values[-1]}% from {df_cpi['Food'].values[-2]}%.")
    elif 'Overall inflation' in df_cpi.columns:
        p = doc.add_heading(f"Inflation {change_cpi['Overall inflation']}", level=2)
        p = doc.add_paragraph(f"Overall inflation {change_cpi['Overall inflation']} to {df_cpi['Overall inflation'].values[-1]}% year-on-year in {calendar.month_name[df_cpi.index[-1].month]} from {df_cpi['Overall inflation'].values[-2]}% in the previous month.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [16]:
if (type(df_policyrate_monthly) is not dict) or (df_policyrate_monthly['codelists']['codesfound']):
    change = {}
    for item in df_policyrate_monthly.columns:
        # Set placeholders
        if df_policyrate_monthly[f'{item}'].values[-1] > df_policyrate_monthly[f'{item}'].values[-2]:
            change[f'{item}'] = random.choice(['tightened', 'increased', 'raised'])
        elif df_policyrate_monthly[f'{item}'].values[-1] == df_policyrate_monthly[f'{item}'].values[-2]:
            change[f'{item}'] = random.choice(['kept', 'maintained', 'held unchanged'])
        else:
            change[f'{item}'] = random.choice(['eased', 'decreased', 'lowered'])

    p = doc.add_paragraph(f"{country_codes.country_name[country]}'s central bank {change['Policy rate (EOP, monthly, %)']} the official policy rate to {df_policyrate_monthly['Policy rate (EOP, monthly, %)'].values[-1]}% in {calendar.month_name[df_policyrate_monthly.index[-1].month]} from {str(df_policyrate_monthly['Policy rate (EOP, monthly, %)'].values[-2]) + '%' if df_policyrate_monthly['Policy rate (EOP, monthly, %)'].values[-1] != df_policyrate_monthly['Policy rate (EOP, monthly, %)'].values[-2] else 'the same'} in {calendar.month_name[df_policyrate_monthly.index[-1].month - 1]}.")
    p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

In [17]:
# # Add a paragraph on Fiscal policy
# p = doc.add_paragraph('Fiscal policy', style='List Number')

# Add a paragraph on Current events
# p = doc.add_paragraph('Comments on current events', style='List Number')

# # Add a paragraph on leading indicators
# p = doc.add_paragraph('Leading indicators', style='List Number')

# df_forecasts = pd.read_csv('consensus_forecasts_13nov.csv', index_col='country')
# df_forecasts = df_forecasts.loc[df_forecasts.index == country].round(1)

# Add a paragraph on Consensus Forecasts
p = doc.add_heading(f"Outlook {'tilted downwards' if country_codes[country_codes.index==country].forecast_gdp_current[0].round(1) < 0 else 'favorable'} this year")
p = doc.add_paragraph(f"On {country_codes[country_codes.index==country].forecast_date[0]}, Consensus Economics panelists project {country_codes.country_name[country]}'s economic growth ending in {country_codes[country_codes.index==country].forecast_date[0][-4:]} at {country_codes[country_codes.index==country].forecast_gdp_current[0].round(1)}%. In {int(country_codes[country_codes.index==country].forecast_date[0][-4:]) + 1}, the panelists foresee growth at {country_codes[country_codes.index==country].forecast_gdp_1step[0].round(1)}%. {'Over the same period, consumption is expected to grow by ' + str(country_codes[country_codes.index==country].forecast_con_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_con_1step[0].round(1)) +'%, while investment is projected to grow by ' + str(country_codes[country_codes.index==country].forecast_inv_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_inv_1step[0].round(1)) +'%.' if ~np.isnan(country_codes[country_codes.index==country].forecast_con_current[0].round(1)) else ''} {'Industrial production is seen to grow by ' + str(country_codes[country_codes.index==country].forecast_ip_current[0].round(1)) + '% and ' + str(country_codes[country_codes.index==country].forecast_ip_1step[0].round(1)) + '%.' if ~np.isnan(country_codes[country_codes.index==country].forecast_ip_current[0].round(1)) else ''}") 
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

p = doc.add_paragraph(f"Meanwhile, Consensus Economics panelists foresee inflation averaging in {country_codes[country_codes.index==country].forecast_date[0][-4:]} at {country_codes[country_codes.index==country].forecast_inf_current[0].round(1)}%. In {int(country_codes[country_codes.index==country].forecast_date[0][-4:]) + 1}, the panelists project inflation at {country_codes[country_codes.index==country].forecast_inf_1step[0].round(1)}%.")
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

# Add a paragraph on Risks
# p = doc.add_paragraph('Risks', style='List Number')

# Add a line for sources
p = doc.add_paragraph()
p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
run = p.add_run(f"Sources: Consensus Economics, The World Bank, UN Comtrade, Haver Analytics, and National Sources. Accessed {dt.date.today().strftime('%d %B %Y')}.")
font = run.font
font.size = Pt(10)
font.italic = True


# Save the .docx file into the local folder
doc.save(f'Country Reports/{country}-Recent-Economic-Developments.docx')